In [1]:
import requests
import json

In [2]:
# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

In [3]:
med_count = {}

for d in data["features"]:
    conunty = d['properties']['county']
    if conunty not in med_count:
        med_count[conunty] = d['properties']["mask_adult"]
    else:
        med_count[conunty] +=d['properties']["mask_adult"]

print(med_count)

{'臺北市': 322490, '': 69210, '高雄市': 424170, '臺中市': 536710, '臺南市': 393900, '基隆市': 80100, '新竹市': 45630, '嘉義市': 107840, '新北市': 670270, '桃園市': 331940, '新竹縣': 51350, '宜蘭縣': 106900, '苗栗縣': 131080, '彰化縣': 219400, '南投縣': 91240, '雲林縣': 196990, '嘉義縣': 113180, '屏東縣': 184250, '澎湖縣': 12600, '花蓮縣': 57330, '臺東縣': 28510, '金門縣': 15510, '連江縣': 7930}


In [4]:
mask_count = dict(sorted(med_count.items(), key=lambda item: item[1], reverse=True))
print(mask_count)

{'新北市': 670270, '臺中市': 536710, '高雄市': 424170, '臺南市': 393900, '桃園市': 331940, '臺北市': 322490, '彰化縣': 219400, '雲林縣': 196990, '屏東縣': 184250, '苗栗縣': 131080, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '': 69210, '花蓮縣': 57330, '新竹縣': 51350, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}


In [5]:
#將資料存入資料庫
import sqlite3
import datetime

conn = sqlite3.connect("example.db")
c = conn.cursor()

#新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
          (city text, counts text, createAt datatime)''')

c.execute('''DELETE FROM pharmacies''')
conn.commit()

#新增資料
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)
d = response.text
data = json.loads(d)
med_count = {}
for d in data["features"]:
    conunty = d['properties']['county']
    if conunty not in med_count:
        med_count[conunty] = d['properties']["mask_adult"]
    else:
        med_count[conunty] +=d['properties']["mask_adult"]
        
mask_count = dict(sorted(med_count.items(), key=lambda item: item[1], reverse=True))

# 將每一筆資料分別取出來新增到資料庫
for key in mask_count:
    city = key
    counts = mask_count[key]
    t = datetime.datetime.now()
    print(f"INSERT INTO stocks VALUES('{city}','{counts}','{t}')")
    c.execute(f"INSERT INTO pharmacies VALUES('{city}','{counts}','{t}')")
    conn.commit()

#查看存入的資料庫
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())
conn.close()

INSERT INTO stocks VALUES('新北市','670270','2024-10-17 10:16:24.938238')
INSERT INTO stocks VALUES('臺中市','536710','2024-10-17 10:16:24.946900')
INSERT INTO stocks VALUES('高雄市','424170','2024-10-17 10:16:24.955449')
INSERT INTO stocks VALUES('臺南市','393900','2024-10-17 10:16:24.958521')
INSERT INTO stocks VALUES('桃園市','331940','2024-10-17 10:16:24.958521')
INSERT INTO stocks VALUES('臺北市','322490','2024-10-17 10:16:24.958521')
INSERT INTO stocks VALUES('彰化縣','219400','2024-10-17 10:16:24.958521')
INSERT INTO stocks VALUES('雲林縣','196990','2024-10-17 10:16:24.971057')
INSERT INTO stocks VALUES('屏東縣','184250','2024-10-17 10:16:24.971057')
INSERT INTO stocks VALUES('苗栗縣','131080','2024-10-17 10:16:24.971057')
INSERT INTO stocks VALUES('嘉義縣','113180','2024-10-17 10:16:24.971057')
INSERT INTO stocks VALUES('嘉義市','107840','2024-10-17 10:16:24.987496')
INSERT INTO stocks VALUES('宜蘭縣','106900','2024-10-17 10:16:24.987496')
INSERT INTO stocks VALUES('南投縣','91240','2024-10-17 10:16:24.987496')
INSERT 